In [1]:
from utils import walls_detection_traditional as wdt
from utils import image_handle as ih
from glob import glob
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import shutil
import imageio

np.random.seed(42)

In [2]:
def plot_image_and_mask(image, mask, mask2):
    plt.figure(figsize=(15, 15))

    plt.subplot(1, 3, 1)
    plt.title('Original Image')
    plt.imshow(image)

    plt.subplot(1, 3, 2)
    plt.title('Mask')
    plt.imshow(mask, cmap='gray')

    plt.subplot(1, 3, 3)
    plt.title('Máscara Final com Linhas Retas')
    plt.imshow(mask2, cmap='gray')

    plt.show()

def draw_segment_on_image(image, label_data):
    
    img_h, img_w = image.shape[:2]  # Dimensões da imagem

    # Exemplo de label no formato: '1 x_center y_center width height x1 y1 x2 y2 ... xn yn'
    data = label_data.strip().split()  # Separar os valores por espaço
    class_id = int(data[0])  # Pegar a classe (aqui está como '1')
    
    # Bounding box normalizada
    x_center, y_center, width, height = map(float, data[1:5])

    # Desnormalizar as coordenadas da bounding box
    x_center *= img_w
    y_center *= img_h
    width *= img_w
    height *= img_h

    # Converter o centro para coordenadas de canto superior esquerdo
    x_min = int(x_center - width / 2)
    y_min = int(y_center - height / 2)
    x_max = int(x_center + width / 2)
    y_max = int(y_center + height / 2)

    # Desenhar a bounding box na cor verde
    #cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

    # Polígono normalizado (x1 y1 x2 y2 ... xn yn)
    polygon_points = np.array([float(coord) for coord in data[5:]]).reshape(-1, 2)

    # Desnormalizar as coordenadas do polígono
    polygon_points[:, 0] *= img_w  # Multiplica as coordenadas X pela largura da imagem
    polygon_points[:, 1] *= img_h  # Multiplica as coordenadas Y pela altura da imagem

    # Converter para inteiros
    polygon_points = polygon_points.astype(np.int32)

    # Desenhar o polígono na cor vermelha
    cv2.polylines(image, [polygon_points], isClosed=True, color=(0, 0, 255), thickness=2)

    return image

def draw_lines_from_labels(image, labels):
    
    img_h, img_w, _ = image.shape  # Altura e largura da imagem

    # Percorrer as labels e desenhar as linhas
    for label in labels:
        cls, x_center, y_center, width, height, x1, y1, x2, y2 = label.split(" ")

        x1_real = int(float(x1) * img_w)
        y1_real = int(float(y1) * img_h)
        x2_real = int(float(x2) * img_w)
        y2_real = int(float(y2) * img_h)

        cv2.line(image, (x1_real, y1_real), (x2_real, y2_real), (0, 255, 0), 3)
        cv2.circle(image, (x1_real, y1_real), 5, (255, 0, 0))
        cv2.circle(image, (x2_real, y2_real), 5, (255, 0, 0))

    return image

In [3]:
files = glob("data/images/*/*")
len(files)

6170

In [4]:
files[0]

'data/images/test/high_quality_1024_F1_original.png'

In [7]:
#df = pd.DataFrame(files, columns=["filename"])
#df = df.filename.str.split("/", expand=True)
#df.columns = ["root", "images", "type", "name", "filename"]
#df["fullfilename"] = df.apply(lambda row: '/'.join(row.values.astype(str)), axis=1)

#n = df.shape[0]
#proportions = ['train'] * int(0.9 * n) + ['val'] * int(0.05 * n) + ['test'] * int(0.05 * n)
#np.random.shuffle(proportions)
#for i in range(n - len(proportions)):
#    proportions.append("train")

#df['split'] = proportions

#df.split.value_counts()

df = pd.DataFrame(files, columns=["filename"])
df = df.filename.str.split("/", expand=True)
df.columns = ["root", "images", "split", "filename"]
df["fullfilename"] = df.apply(lambda row: '/'.join(row.values.astype(str)), axis=1)
df

,root,images,split,filename,fullfilename
0,data,images,test,high_quality_1024_F1_original.png,data/images/test/high_quality_1024_F1_original...
1,data,images,test,high_quality_133_F1_original.png,data/images/test/high_quality_133_F1_original.png
2,data,images,test,high_quality_architectural_5973_F1_original.png,data/images/test/high_quality_architectural_59...
3,data,images,test,colorful_1510_F2_original.png,data/images/test/colorful_1510_F2_original.png
4,data,images,test,high_quality_architectural_3697_F1_original.png,data/images/test/high_quality_architectural_36...
...,...,...,...,...,...
6165,data,images,val,high_quality_architectural_6158_F1_original.png,data/images/val/high_quality_architectural_615...
6166,data,images,val,high_quality_269_F1_original.png,data/images/val/high_quality_269_F1_original.png
6167,data,images,val,high_quality_404_F1_original.png,data/images/val/high_quality_404_F1_original.png
6168,data,images,val,high_quality_architectural_4189_F1_original.png,data/images/val/high_quality_architectural_418...


In [12]:
for i, row in tqdm(df.iterrows(), total=6170):

    image_original = cv2.cvtColor(cv2.imread(row.fullfilename), cv2.COLOR_BGR2RGB)
    image = cv2.imread(row.fullfilename, cv2.IMREAD_GRAYSCALE)
    mask = wdt.create_mask(image, 0.4)
    mask_straight, labels = wdt.get_straight_lines(mask)

    new_filename = f"data/images/{row.split}/{row['type']}_{row.name}_{row.filename}"
    shutil.copy(row.fullfilename, new_filename)

    labelname = f"data/labels/{row.split}/{row.filename.split('.')[0]}.txt"
    with open(labelname, 'w') as file:
        for label in labels:
            file.write(f"{label}\n")

100%|███████████████████████████████████████| 6170/6170 [06:22<00:00, 16.13it/s]
